In [1]:
import json
import requests

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [3]:
def submit_to_ars(m):
    submit_url = 'https://ars.transltr.io/ars/api/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'https://arax.ncats.io/?source=ARS&id={message_id}')
    return message_id

In [4]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        if child['status'] == 'Done':
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [5]:
def printjson(j):
    print(json.dumps(j,indent=4))

def print_json(j):
    printjson(j)

In [6]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [7]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

# Look at processes?

In [14]:
qg = {
  "nodes": {
    "n0": {
      "name": "Glycerol",
      "id": "PUBCHEM.COMPOUND:753"
    },
    "n1": {
      "category": [
        "biolink:BiologicalProcessOrActivity"
      ],
      "name": "Biological Process Or Activity"
    },
    "n2": {
      "id":"NCBIGene:2710"
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1",
      "predicate": [
        "biolink:related_to"
      ]
    },
    "e1": {
      "subject": "n1",
      "object": "n2",
      "predicate": [
        "biolink:related_to"
      ]
    }
  }
}

process_query = {'message':{'query_graph':qg}}

In [15]:
glycerol_process_id = submit_to_ars(process_query)

https://arax.ncats.io/?source=ARS&id=3f3cdaa4-ff33-43b6-b541-9e711e9a0853


In [17]:
results = retrieve_ars_results(glycerol_process_id)

Done
Done ara-arax 1
Done ara-aragorn 0
Error ara-bte 0
Done ara-unsecret 0
Done kp-genetics 0
Error kp-molecular 0
Done ara-explanatory 0
Done ara-improving 0
Error kp-cam 0
Error kp-textmining 0
Done ara-aragorn-exp 0
Error kp-openpredict 0
Done kp-icees 0
Unknown ara-ncats 0
Unknown ara-robokop 0
Error kp-chp 0


In [18]:
results['ara-arax']

{'message': {'knowledge_graph': {'edges': {'ARAX/KG2:CHEMBL.COMPOUND:CHEMBL692-biolink:affects-MESH:D004734': {'attributes': [{'name': 'provided_by',
       'type': 'biolink:provided_by',
       'value': ['SEMMEDDB:']},
      {'name': 'publications',
       'type': 'biolink:publications',
       'value': ['PMID:3041641', 'PMID:3304183', 'PMID:889892']},
      {'name': 'is_defined_by',
       'type': 'biolink:Unknown',
       'value': 'ARAX/KG2'}],
     'object': 'MESH:D004734',
     'predicate': 'biolink:affects',
     'subject': 'CHEMBL.COMPOUND:CHEMBL692'},
    'ARAX/KG2:CHEMBL.COMPOUND:CHEMBL692-biolink:affects-MESH:D011485': {'attributes': [{'name': 'provided_by',
       'type': 'biolink:provided_by',
       'value': ['SEMMEDDB:']},
      {'name': 'publications',
       'type': 'biolink:publications',
       'value': ['PMID:606543']},
      {'name': 'is_defined_by',
       'type': 'biolink:Unknown',
       'value': 'ARAX/KG2'}],
     'object': 'MESH:D011485',
     'predicate': 'bio

In [19]:
view = GammaViewer(props={"data":results['ara-arax']})
display(view)

In [20]:
qg = {
  "nodes": {
    "n0": {
      "name": "Glycerol",
      "id": "PUBCHEM.COMPOUND:753"
    },
    "n1": {
      "category": [
        "biolink:BiologicalProcessOrActivity"
      ],
      "name": "Biological Process Or Activity"
    },
    "n2": {
      "id":"NCBIGene:2710"
    } ,
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1"
    },
    "e1": {
      "subject": "n1",
      "object": "n2"
    }
  }
}

process_query_no_predicates = {'message':{'query_graph':qg}}

In [22]:
glycerol_process_nopreds_id = submit_to_ars(process_query_no_predicates)

https://arax.ncats.io/?source=ARS&id=3003b8ab-4f8b-4fce-ae47-a3088876aea8


In [23]:
qg = {
  "nodes": {
    "n0": {
      "name": "Glycerol",
      "id": "PUBCHEM.COMPOUND:753"
    },
    "n1": {
      "category": [
        "biolink:Pathway"
      ]
    },
    "n2": {
      "id":"NCBIGene:2710"
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1"
    },
    "e1": {
      "subject": "n1",
      "object": "n2"
    }
  }
}

pathway_query_no_predicates = {'message':{'query_graph':qg}}

In [24]:
glycerol_pathway_nopreds_id = submit_to_ars(pathway_query_no_predicates)

https://arax.ncats.io/?source=ARS&id=8321c67c-4824-4f2a-8605-d16470a45f43


In [25]:
qg = {
  "nodes": {
    "n0": {
      "name": "Glycerol",
      "id": "PUBCHEM.COMPOUND:753"
    },
    "n1": {
      "category": [
        "biolink:BiologicalProcessOrActivity"
      ]
    },
    "n2": {
      "id":"NCBIGene:2710"
    } ,
    "n3": {
        "id": "MONDO:0010613"
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1"
    },
    "e1": {
      "subject": "n1",
      "object": "n2"
    },
      "e2": {
      "subject": "n1",
      "object": "n3"
    }
  }
}

process_query_G_GK_GKD = {'message':{'query_graph':qg}}

In [26]:
star_id = submit_to_ars(process_query_G_GK_GKD)

https://arax.ncats.io/?source=ARS&id=f72a9b1c-d829-4834-9596-0cb8498f4237
